<h2><center>
    Loading In Data
    </center></h2>

In [180]:
from github import Github
import os
import pickle
import base64
import pandas as pd
import datetime as dt
from os import listdir
from os.path import isfile, join
import numpy as np
mypath = 'data_csse/'

In [181]:
with open("github.txt") as myfile:
    firstNlines=myfile.readlines()[0:2]
myfile.close()
g = Github(firstNlines[0].strip(), firstNlines[1])
repo=g.get_repo('CSSEGISandData/COVID-19')
contents = repo.get_contents("")

In [182]:
def get_sha_for_tag(repository, tag):
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [183]:
def download_directory(repository, sha, server_path, local_path=mypath):
    contents = repository.get_contents(server_path, ref=sha)
    if not os.path.exists(local_path):
        os.makedirs(local_path)
    for content in contents:
        if content.type == 'dir':
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                file_data = base64.b64decode(file_content.content).decode('ascii')
                file_out = open(local_path+content.name, "w")
                file_out.write(local_path+file_data)
                file_out.close()
            except:
                pass

In [184]:
def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [185]:
def preprocess_df(df, name):
    df.drop(columns=['data_csse/Province/State', 'Lat', 'Long'], inplace=True)
    df = df.groupby(['Country/Region']).agg('sum')
    df = df.transpose().reset_index()
    country_list = list(df.columns)[1:]
    df = pd.melt(df, id_vars='index', value_vars=country_list)
    df = df.rename(columns={'index':'Date', 'value':name})
    return df

In [186]:
sha = get_sha_for_tag(repo, 'master')
download_directory(repo, sha, 'csse_covid_19_data/csse_covid_19_time_series/')

In [187]:
#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# timeseries_files = []
# for file in onlyfiles:
#     if is_non_zero_file(mypath + file) and file[-10:]=='global.csv':
#         timeseries_files.append(mypath + file)

In [188]:
confirmed_df = pd.read_csv('data_csse/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('data_csse/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('data_csse/time_series_covid19_recovered_global.csv')

In [189]:
confirmed_df = preprocess_df(confirmed_df, 'Confirmed')
deaths_df = preprocess_df(deaths_df, 'Deaths')
recovered_df = preprocess_df(recovered_df, 'Recovered')

<h2><center>
    Preprocessing Data
    </center></h2>

In [190]:
confirmed_and_deaths = pd.merge(confirmed_df, deaths_df, how='inner', on=['Date', 'Country/Region'])
grouped_df = pd.merge(confirmed_and_deaths, recovered_df, how='inner', on=['Date', 'Country/Region'])
grouped_df['Active'] = grouped_df['Confirmed']-grouped_df['Deaths']-grouped_df['Recovered']
grouped_df['Datetime'] = grouped_df['Date'].apply(lambda x: pd.to_datetime(x))

In [191]:
pop_df = pd.read_csv('pop_df4.csv')
pop_df['Country/Region'] = pop_df['Country/Region'].apply(lambda x: x.replace('Mainland China', 'China'))
pop_df.drop(columns=['Unnamed: 0'], inplace=True)

In [192]:
country_list=[('Mainland China', 'China'),
('Hong Kong SAR', 'Hong Kong'),
(['Korea, South', 'Republic of Korea'], 'South Korea'),
('United Kingdom', 'UK'),
(['Taiwan*', 'Taipei and environs'], 'Taiwan'),
('Iran (Islamic Republic of)', 'Iran'),
('Viet Nam', 'Vietnam'),
('Macao SAR', 'Macau'),
('Republic of Ireland', 'Ireland'),
('Czechia', 'Czech Republic'),
('occupied Palestinian territory', 'Palestine'),
('Russian Federation', 'Russia'),
(' Azerbaijan', 'Azerbaijan'),
('Holy See', 'Vatican City'),
('Republic of Moldova', 'Moldova'),
('Saint Martin', 'St. Martin')]

In [193]:
for name in country_list:
    grouped_df['Country/Region']=grouped_df['Country/Region'].replace(name[0],name[1])

In [194]:
if not 'PopTotal' in set(grouped_df.columns):
    grouped_df = pd.merge(grouped_df, pop_df, on='Country/Region')
grouped_df['Confirmed Cases Per 1M'] = grouped_df['Confirmed']*1000/grouped_df['PopTotal']
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('US', 'United States')

In [195]:
grouped_df['New Weekly Cases'] = np.where(grouped_df['Country/Region']==grouped_df['Country/Region'].shift(7),
                                          grouped_df['Confirmed']-grouped_df['Confirmed'].shift(7),grouped_df['Confirmed'])

<h2><center>
    Export Dataframe
    </center></h2>

In [196]:
with open('COVID_Hopkins_df.pickle', 'wb') as pickle_out:
    pickle.dump(grouped_df, pickle_out)

In [197]:
with open('../covid19app/COVID_Hopkins_df.pickle', 'wb') as pickle_out:
    pickle.dump(grouped_df, pickle_out)

In [198]:
grouped_df.tail()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,Datetime,PopTotal,Confirmed Cases Per 1M,New Weekly Cases
11213,3/28/20,Zimbabwe,7,1,0,6,2020-03-28,14862.927,0.470970,4.0
11214,3/29/20,Zimbabwe,7,1,0,6,2020-03-29,14862.927,0.470970,4.0
11215,3/30/20,Zimbabwe,7,1,0,6,2020-03-30,14862.927,0.470970,4.0
11216,3/31/20,Zimbabwe,8,1,0,7,2020-03-31,14862.927,0.538252,5.0
11217,4/1/20,Zimbabwe,8,1,0,7,2020-04-01,14862.927,0.538252,5.0


In [199]:
grouped_df.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,Datetime,PopTotal,Confirmed Cases Per 1M,New Weekly Cases
0,1/22/20,Afghanistan,0,0,0,0,2020-01-22,38928.341,0.0,0.0
1,1/23/20,Afghanistan,0,0,0,0,2020-01-23,38928.341,0.0,0.0
2,1/24/20,Afghanistan,0,0,0,0,2020-01-24,38928.341,0.0,0.0
3,1/25/20,Afghanistan,0,0,0,0,2020-01-25,38928.341,0.0,0.0
4,1/26/20,Afghanistan,0,0,0,0,2020-01-26,38928.341,0.0,0.0
